In [1]:
from typing import List, Tuple

import numpy as np
import torch
import torchvision
from torch import nn
from torch.nn.functional import cross_entropy
from torch.optim import SGD
from torchvision.datasets import MNIST

# Sieci neuronowe

## MNIST
Popularnym "prostym" datasetem, na którym można przetestować nasz model zanim zajmiemy się trudniejszymi problemami, jest MNIST, zbiór danych zawierających ręcznie rysowane cyfry. Poniżej kilka przykładowych cyfr:

<img width="400" src="https://miro.medium.com/proxy/0*At0wJRULTXvyA3EK.png" />


Zadanie naszego modelu polega na tym, by na podstawie obrazka narysowanej ręcznie cyfry określić, jaka to jest cyfra.

Czyli nasz model, widząc taki obrazek:
<img width="200" src="https://machinelearningmastery.com/wp-content/uploads/2019/02/sample_image-768x763.png" />

powinien odpowiedzieć, że to cyfra "7".


**Pytanie: Czy w takim sformułowaniu MNIST służy do regresji czy do klasyfikacji?**

## Praca na obrazkach
Nasze modele jak dotąd przyjmowały wyłącznie wektory - w jaki sposób możemy w takich modelach przetwarzać obrazki?

Obrazek to tak naprawdę trójwymiarowa tablica pikseli o wymiarach: `[H, W, C]`, gdzie `H` to wysokość, `W` to szerokość, a `C` to liczba kanałów (klasycznie: red, green, blue).

Najprostsze co możemy zrobić, to spłaszczyć naszą tablicę do jednego wymiaru, wektora o kształcie `[H * W * C]`. W przypadku MNIST-a nasze obrazki mają wymiar `28x28` pikseli i jest tylko jeden kanał (odcienie szarości), więc każdy z naszych wektorów będzie miał kształt `[28 * 28] = [784]`.

W przyszłości poznamy też sprytniejsze sposoby działania na obrazkach, np. za pomocą sieci konwolucyjnych.

## Stochastic gradient descent

Dotychczas kiedy chcieliśmy minimalizować funkcję kosztu $L(X; \theta)$ dla całego naszego zbioru $X \in \mathbb{R}^{NxD}$, liczyliśmy średni koszt dla wszystkich elementów $x \in X$, tzn. 

$$L(X; \theta) = \frac{1}{N} \sum_i L(x_i; \theta) $$

Następnie liczyliśmy gradient tego kosztu, żeby zminimalizować funkcję.

W praktyce może się okazać, że nasz dataset jest gigantyczny, np. kiedy mamy miliony przykładów. Niepraktyczne wtedy jest liczenie całego tego kosztu a tym bardziej gradientu. W praktyce w każdym kroku liczymy funkcję kosztu (i jej gradient) z innego podzbioru elementów w naszym zbiorze, czyli z tzw. **batcha**:

$$L_{\mathrm{batch}} (X;\theta) = \frac{1}{|B|} \sum_{x \in B} L(x; \theta) $$ 

Gradient po koszcie policzonym z batcha będzie inny niż gradient liczony po koszcie policzonym z całego zbioru, ale powinny być w miarę podobne, tzn:

$$ \nabla_\theta L_{\mathrm{batch}} (X; \theta) \approx \nabla_\theta L(X; \theta) $$

Metodę spadku gradientu zaimplementowaną w ten sposób (batchowo) nazywamy metodą **stochastycznego spadku gradientu** (*Stochastic Gradient Descent, SGD*). 


## Torchvision
PyTorch, a także pakiet `torchvision` udostępnia parę przydatnych narzędzi, z których skorzystamy na dzisiejszych zajęciach. Dla przykładu znacznie uproszczone jest pobieranie i ładowanie danych. W pakiecie [`torchvision.datasets`](https://pytorch.org/docs/stable/torchvision/datasets.html) znajdziemy popularne datasety, m.in. właśnie MNIST-a.

Oprócz tego z samego `torcha` możemy skorzystać z [`DataLoadera`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader), który implementuje wiele przydatnych operacji do ładowania danych, np. dzielenie datasetu w batche i shufflowanie.

## Zadanie 1 (2 pkt.)
Klasa `MNIST` zwraca nam dane w postaci obiektów [PIL](https://pillow.readthedocs.io/en/stable/). Należy je odpowiednio przetworzyć, zanim będziemy mogli na nich pracować.

Za pomocą [`transformacji`](https://pytorch.org/vision/stable/transforms.html) podawanych do klasy MNIST należy:
1. Zamienić obiekty `PIL` na Tensory.
2. Policzyć średnią i odchylenie standardowe pikseli dla **całego zbioru trenującego** i użyć ich później do znormalizowania danych trenujących i testowych. Do liczenia średniej i odchylenia standardowego wykorzystać funkcję  `calculate_mean_and_std` (proszę zwrócić uwagę na to w jakim przedziale znajdują się dane przed normalizacją – chcemy aby były w przedziale 0-1). **HINT**: Tutaj torchvision powinien nam to ułatwić.
3. Zmienić "kształt" każdego przykładu z `28x28` na `784`.
    **HINT**: [`Lambda`](https://pytorch.org/vision/stable/generated/torchvision.transforms.Lambda.html)

Uwaga: proszę zwrócić uwagę co dokładnie robią używane_transformacje!

In [7]:
from torchvision.transforms import Compose, ToTensor, Normalize, Lambda


def calculate_mean_and_std() -> Tuple[float, float]:
    tmp_dataset = MNIST(root=".", download=True, train=True, transform=ToTensor())
    all_tensors = []
    for img, _ in tmp_dataset:
        # img ma  [1, 28, 28], a value w [0,1]
        all_tensors.append(img)
    all_tensors = torch.stack(all_tensors, dim=0)  # [N,1,28,28]
    mean = all_tensors.mean().item()
    std = all_tensors.std().item()
    return mean, std

mean, std = calculate_mean_and_std()
train_transform = Compose([
    ToTensor(),                       #  PIL -> Tensor ( 0..1)
    Normalize((mean,), (std,)),      #  ~ N(0,1)
    Lambda(lambda x: x.view(-1))     #   [1,28,28] -> [784]
])

train_data = MNIST(
    root=".",
    download=True,
    train=True,
    transform = train_transform
)

test_data = MNIST(
    root=".",
    download=True,
    train=False,
    transform = train_transform

)

In [8]:
mean, std = calculate_mean_and_std()
assert np.isclose(mean, 0.1306, atol=1e-4)
assert np.isclose(std, 0.3081, atol=1e-4)

In [11]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=10)

x, y = next(iter(train_loader))
assert len(x.shape) == 2
assert x.shape == (10, 784)

# Sieci neuronowe

### Modele liniowe

Jak dotąd omawialiśmy wyłącznie modele liniowe, tzn. takie, które dla zadanego $x$ potrafiły modelować funkcję rodzaju $$f(x) = g(w^T x + b) $$
gdzie $x \in \mathbb{R}^D, w \in \mathbb{R}^D$, $b \in \mathbb{R}$ a $g$ to funkcja aktywacji, np. sigmoid.

Możemy też stworzyć podobny model, który na wyjściu nie będzie podawał jednej liczby, ale cały wektor o wymiarze $K$, tzn:
$$f(x) = g(W^T x + \mathbf{b}), $$
gdzie $W$ jest teraz macierzą a $\mathbf{b}$ wektorem, tzn. $W \in \mathbb{R}^{DxK}, \mathbf{b} \in \mathbb{R}^{K}$.

### Zanurzenia

Jeżeli chcieliśmy, żeby takie modele mogły zajmować się problemami nieliniowymi, musieliśmy znaleźć odpowiednią reprezentację danych (zanurzenia wielomianowe, kernele dla SVM-ów), które sprawi, że w nowej przestrzeni problem będzie liniowy. W tym celu trzeba "zgadnąć", jakie przekształcenie jest właściwe - co w przypadku bardziej skomplikowanych problemów jest niezwykle trudne. 

Ważne pytanie: **czy jesteśmy w stanie zbudować model, który znajdzie nam odpowiednią reprezentację dla danych?**

### Nakładanie warstw liniowych

**Rozwiązanie:** Nałóżmy na siebie kilka warstw modeli liniowych, np:
$$
f(x) = g_2(W_2^T (g_1(W_1^T x + \mathbf{b_1})) + \mathbf{b_2}),
$$
czyli, rozpisując czytelniej:
$$
f(x) = f^{(2)}(f^{(1)}(x)) \\
f^{(1)}(x) = g_1(W_1^T x + \mathbf{b_1}) \\ 
f^{(2)}(x) = g_2(W_2^T x + \mathbf{b_2})
$$

Powstały model nazywamy **sztuczną siecią neuronową** (*artificial neural network*).
 
Każdą funkcję $f^{(i)}$ nazywamy **warstwą** (*layer*). W naszej sieci możemy umieścić dowolnie wiele warstw, ale na razie będziemy zajmować się modelami nieszczególnie głębokimi (mniej niż 10 warstw).

Warstwy $f^{(i)}$ mogą implementować dowolną funkcję, ale jeśli mają postać $g(W^Tx +\mathbf{b})$, to nazywamy je warstwami liniowymi lub warstwami *fully connected*. Na tych zajęciach będziemy zajmować się wyłącznie sieciami o takiej postaci.


### Uczenie się reprezentacji

Jeśli nasz model jest postaci
$$
f(x) = f^{(n)}(f^{(n-1)}(\ldots f^{1}(x) \ldots )),
$$
to możemy przyjąć, że warstwa $f^{(n)}$ rozwiązuje problem liniowy na reprezentacji zadanej przez warstwy $f^{(1)}, f^{(2)}, \ldots, f^{(n-1)}$. 



<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/46/Colored_neural_network.svg/1200px-Colored_neural_network.svg.png" width=300 />  

Źródło: [Wikipedia](https://en.wikipedia.org/wiki/Artificial_neural_network).

### Neurony

**Neuron** w tym kontekście to fragment warstwy, który łączy się ze wszystkimi neuronami w poprzedniej warstwie i na ich podstawie produkuje jedno z wyjść warstwy. 

Jeśli nasza warstwa ma postać $g(W^T x + \mathbf{b})$, to $i$-ty neuron implementuje funkcję:
$$ f(x) = g(w_i^T x + b_i), $$
gdzie wektor $w_i$ jest $i$-tą kolumną macierzy $W$ a $b_i$ jest $i$-tym elementem wektora $\mathbf{b}$.

## Zadanie 2 (3 pkt.)
Za pomocą przygotowanego przez TensorFlow [narzędzia do zabawy z sieciami neuronowymi](http://playground.tensorflow.org) proszę przeprowadzić poniższe eksperymenty i opisać rezultaty.

Kilka uwag:
* Każda odpowiedź powinna być zawarta w jednym/dwóch zdaniach. 
* Punktowane będą nie tylko prawidłowe odpowiedzi, ale też sensowne hipotezy/przypuszczenia.
* Jeżeli proszeni są Państwo o podanie architektury sieci, to najlepiej zapisać ją w skrótowe postaci `n_1-n_2-...-n_k`, gdzie `n_i` to liczba neuronów w `i`-tej warstwie. Czyli jeśli sieć ma pięć neuronów w pierwszej warstwie, trzy neurony w drugiej warstwie oraz sześć neuronów w trzeciej warstwie, to można ją opisać jako `5-3-6`.
* Proszę nie zmieniać opcji noise/ratio of training to test data. Proszę nie zmieniać feature'ów wejściowych, o ile nie będzie to wyraźnie podane w zadaniu.
* Można użyć opcji "show test data", żeby sprawdzić, dlaczego koszt na datasecie treningowym i testowym się różni.

1) **Eksperymenty na zbiorze Gaussian**
* Czy ten dataset można rozwiązać metodami płytkimi, których uczyliśmy się na wcześniejszych zajęciach?
* Co sprawia, że ten zbiór danych jest łatwiejszy niż pozostałe?
* Porównaj na tym zbiorze dwa modele: sieć neuronową z kilkoma warstwami i kilkudziesięcioma neuronami oraz sieć z jednym neuronem. Który z tych modeli bardziej nadaje się do zadania?

In [ ]:
# Tak – zbiór Gaussian da się w łatwy sposób rozdzielić metodą np. regresji logistycznej.
# Ponieważ granica decyzyjna jest praktycznie kołowa (lub lekko eliptyczna)
# Jeden neuron (z aktywacją nieliniową) nie jest w stanie uzyskać granicy okręgu. 
# Duża sieć z kilkoma warstwami i np. kilkunastoma/kilkudziesięcioma neuronami 
# potrafi się w pełni dopasować i rozróżnić okrąg oraz środek,

2) **Eksperymenty na zbiorze Circle**
* Załóżmy, że mamy sieć z jednym neuronem. Ile najmniej potrzeba feature'ów wejściowych, żeby model osiągał na datasecie testowym koszt $\leq 0.001$? Jakie to feature'y?
* Załóżmy, że na wejściu mamy tylko niezanurzone feature'y (tzn. $x_1$ oraz $x_2$). Stwórz najmniejszą sieć neuronową (pod względem liczby neuronów), która osiąga na datasecie testowym koszt $\leq 0.001$. Opisz architekturę tej sieci. 
* Spróbuj rozwiązać ten problem za pomocą dowolnie dużej sieci neuronowej **z aktywacjami liniowymi** (nie zmieniając feature'ów wejściowych). Czy udało się osiągnąć wynik $\leq 0.001$? Jeśli tak, podaj architekturę sieci. Jeśli nie, podaj hipotezę, dlaczego się nie udało.

In [6]:
# Potrzebujemy przynajmniej 1 cechy R = x**2 + y**2 ,Dzięki temu jeden neuron może rozróżnić okrąg, osiągając koszt <0.001.
# Wystarczy sieć z jedną warstwą ukrytą, np. 4-2, z aktywacją ReLU lub tanh. Taka architektura może osiągnąć koszt <0.001 na teście.
# Nie udało się osiągnąć kosztu <0.001, ponieważ sieć z liniowymi aktywacjami nie potrafi rozdzielić okręgu. Nawet wiele warstw to nadal jedna funkcja liniowa.



3) **Eksperymenty na zbiorze Spiral**
* Osiągnij (stabilny) koszt $\leq 0.1$ na zbiorze testowym, podaj wykorzystaną architekturę, rodzaj aktywacji, regularyzację  oraz learning rate.
* Co odróżnia rozwiązania które dobrze generalizują od rozwiązań, które overfitują pod względem wizualnym? Popatrz na płaszczyznę z danymi po wytrenowaniu modelu.

In [7]:
# Architektura: 8-8-8 (trzy warstwy, po 8 neuronów każda)
# Aktywacja: ReLU lub Tanh
# Regularyzacja: L2 = 0.001
# Learning Rate: 0.03
# Koszt na teście: ~0.05–0.08

# --

# Overfitting: Skomplikowana granica decyzyjna, dobrze działa na treningu, źle na teście.
# Dobre generalizowanie: Gładka granica decyzyjna, dobrze działa zarówno na treningu, jak i na teście.


## Zadanie 3 (2 pkt.)

Ręcznie zaimplementować prostą sieć z jedną warstwą ukrytą. Sieć:
1. Na wejściu będzie przyjmować dane o wymiarze `input_dim`
2. Pierwsza warstwa ma je przetwarzać na wymiar `hidden_dim`.
3. Druga warstwa ma przetwarzać wyjście pierwszej warstwy na wymiar `output_dim`.

W tym celu trzeba stworzyć odpowiednie tensory reprezentujące wagi i biasy w poszczególnych warstwach
1. Macierze wag należy zainicjalizować za pomocą wartości wylosowanych ze standardowego rozkładu normalnego. 
2. Dla obu warstw należy stworzyć _biasy_ zainicjalizowane na 0.
3. Funkcją aktywacji dla pierwszej warstwy ma być `torch.tanh`. W drugiej warstwie ma być aktywacja liniowa (czyli brak aktywacji).

Następnie należy zaimplementować pętlę uczenia z użyciem PyTorchowej funkcji kosztu `nn.CrossEntropyLoss` i optymalizatora SGD. Jeśli wszystko zostało zaimplementowane poprawne, to sieć powinna zazwyczaj osiagać accuracy większe niż `0.82` na zbiorze testowym (chociaż czasami może nie osiągać tej wartości z powodu pechowej inicjalizacji).

**HINT** Proszę nie zapomnieć o `requires_grad=True` przy definiowaniu parametrów sieci.

In [15]:
class CustomNetwork(object):
    """
    Simple 1-hidden layer linear neural network
    """

    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int):
        """
        Initialize network's weights
        """

        self.weight_1: torch.Tensor = torch.randn(input_dim, hidden_dim, requires_grad=True)
        self.bias_1: torch.Tensor = torch.zeros(hidden_dim, requires_grad=True)

        self.weight_2: torch.Tensor = torch.randn(hidden_dim, output_dim, requires_grad=True)
        self.bias_2: torch.Tensor = torch.zeros(output_dim, requires_grad=True)

    def __call__(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass through the network
        """

        # W 1
        z1 = x.mm(self.weight_1) + self.bias_1  # shape [batch, hidden_dim]
        a1 = torch.tanh(z1)

        # W 2 (linear)
        z2 = a1.mm(self.weight_2) + self.bias_2
        return z2  # shape [batch, output_dim]

    def parameters(self) -> List[torch.Tensor]:
        """
        Returns all trainable parameters
        """
        return [self.weight_1, self.bias_1, self.weight_2, self.bias_2]

In [17]:
def pytorch_backward(loss: torch.Tensor, model: CustomNetwork):
    loss.backward()

In [94]:
def train(model: CustomNetwork, epoch: int, batch_size: int, lr: float, momentum: float, backward_fn=pytorch_backward):
    correct = 0
    # prepare data loaders, based on the already loaded datasets
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

    # initialize the optimizer using the hyperparams
    optimizer: torch.optim.Optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)

    criterion = nn.CrossEntropyLoss()

    # training loop
    for e in range(epoch):
        for i, (x, y) in enumerate(train_loader):
            # reset the gradients from previouis iteration
            optimizer.zero_grad()
            # pass through the network
            output = model(x)  # type: torch.Tensor
            # calculate loss
            loss = criterion(output, y)  # type: torch.Tensor
            # backward pass thorught the network
            backward_fn(loss, model)
            # apply the gradients
            optimizer.step()

            # log the loss value
            if (i + 1) % 100 == 0:
                print(
                    f"Epoch {e} iter {i + 1}/{len(train_data) // batch_size} loss: {loss.item()}",
                    end="\r",
                )

        # at the end of an epoch run evaluation on the test set
        with torch.no_grad():
            # initialize the number of correct predictions
            correct: int = 0
            for i, (x, y) in enumerate(test_loader):
                # pass through the network
                output = model(x)  # Прямой проход
                _, preds = torch.max(output, dim=1)  # Получаем предсказанные классы
                correct += (preds == y).sum().item()  # Сравниваем с истинными метками

            print(f"\nTest accuracy: {correct / len(test_data)}")
    return correct

In [96]:
# hyperparams
batch_size: int = 64
epoch: int = 2
lr: float = 0.01
momentum: float = 0.9

# initialize the model
model: CustomNetwork = CustomNetwork(784, 64, 10)  # type: ignore

correct = train(model, epoch, batch_size, lr, momentum)

# this is your test
assert (
    correct / len(test_data) > 0.82
), "Subject to random seed you should be able to get >82% accuracy"

Epoch 0 iter 900/937 loss: 0.4711691737174988
Test accuracy: 0.8012
Epoch 1 iter 900/937 loss: 0.2791307866573334
Test accuracy: 0.8357


## Zadanie 4 (2 pkt.)

Ręcznie zaimplementować backward pass do sieci z poprzedniego zadania

It need older version of pytorch

In [118]:
def my_manual_backward(loss: torch.Tensor, model: CustomNetwork):
    with torch.no_grad():
        # Идём по цепочке backward-операций, чтобы достать сохранённые тензоры:
        out_grad = torch.ones_like(loss)                       # т.к. dL/dL = 1
        nll_grad_fn = loss.grad_fn                             # NLLLossBackward
        log_softmax_grad_fn = nll_grad_fn.next_functions[0][0] # LogSoftmaxBackward
        add_grad_fn = log_softmax_grad_fn.next_functions[0][0] # AddBackward0 (слагает z2 = a1@W2 + b2)
        x_times_w2_grad_fn = add_grad_fn.next_functions[0][0]  # MmBackward (a1@W2)
        tanh_grad_fn = x_times_w2_grad_fn.next_functions[0][0] # TanhBackward (a1 = tanh(z1))
        x_times_w1_grad_fn = tanh_grad_fn.next_functions[0][0].next_functions[0][0] 
        # (ещё один MmBackward, соответствующий x@W1 + b1)
        
        # 1) Градиент NLLLoss wrt log_probs
        # nll_grad_fn хранит (log_probs, target) в saved_tensors
        log_probs, target = nll_grad_fn.saved_tensors
        batch_size = target.shape[0]
        
        nll_grad = torch.zeros_like(log_probs)  # dL/d(log_probs)
        nll_grad[torch.arange(batch_size), target.long()] = -1.0 / batch_size
        nll_grad *= out_grad  # out_grad = 1, но умножаем для порядка
        
        # 2) Градиент log_softmax wrt входу log_softmax (т.е. z2)
        probs = log_probs.exp()  # softmax(z2)
        sum_ = (probs * nll_grad).sum(dim=1, keepdim=True)
        log_softmax_grad = nll_grad - probs * sum_
        
        # 3) Градиенты wrt (a1, b2)
        # x_times_w2_grad_fn.saved_tensors = (a1, W2, b2)
        a1, w2, b2 = x_times_w2_grad_fn.saved_tensors
        w2_times_x_grad = log_softmax_grad.mm(w2.t())  # dL/da1
        b2_grad = log_softmax_grad                    # dL/db2 (суммируем позже)
        
        b2_grad = b2_grad.sum(0)  # 4) sum по batch -> градиент wrt b2
        
        # 5) Градиенты wrt (x, W2)
        x_grad = w2_times_x_grad                       # dL/d(a1) передаём дальше
        w2_grad = a1.t().mm(log_softmax_grad)          # dL/dW2
        
        # 6) Градиент tanh wrt z1
        # tanh_grad_fn.saved_tensors = (z1,)
        z1, = tanh_grad_fn.saved_tensors
        a1 = torch.tanh(z1)  # восстанавливаем a1
        tanh_grad = (1 - a1**2) * x_grad  # dL/dz1 = (1 - a1^2) * dL/da1
        
        # 7) Градиенты wrt (x, b1)
        # x_times_w1_grad_fn.saved_tensors = (x_, W1, b1)
        x_, w1, b1 = x_times_w1_grad_fn.saved_tensors
        w1_times_x_grad = tanh_grad   # dL/dx (не нужен дальше), dL/db1 = то же (суммируем ниже)
        b1_grad = w1_times_x_grad
        
        b1_grad = b1_grad.sum(0)  # 8) sum по batch -> градиент wrt b1
        
        # 9) Градиент wrt W1
        w1_grad = x_.t().mm(tanh_grad)  # dL/dW1
        
        # Финальное присвоение градиентов нужным параметрам сети
        model.weight_1.grad = w1_grad
        model.bias_1.grad   = b1_grad
        model.weight_2.grad = w2_grad
        model.bias_2.grad   = b2_grad


In [124]:
model: CustomNetwork = CustomNetwork(784, 64, 10)
(x, y) = next(iter(train_loader))
output = model(x)
loss = torch.nn.functional.cross_entropy(output, y)


my_manual_backward(loss, model)

w1g = model.weight_1.grad.clone().detach()
b1g = model.bias_1.grad.clone().detach()
w2g = model.weight_2.grad.clone().detach()
b2g = model.bias_2.grad.clone().detach()

model.weight_1.grad = None
model.bias_1.grad = None
model.weight_2.grad = None
model.bias_2.grad = None

loss.backward()

assert torch.allclose(w1g, model.weight_1.grad)
assert torch.allclose(b1g, model.bias_1.grad)
assert torch.allclose(w2g, model.weight_2.grad)
assert torch.allclose(b2g, model.bias_2.grad)

AttributeError: 'MmBackward0' object has no attribute 'saved_tensors'

In [126]:
# initialize the model
model: CustomNetwork = CustomNetwork(784, 64, 10)  # type: ignore

correct = train(model, epoch, batch_size, lr, momentum, backward_fn=my_manual_backward)

# this is your test
assert (
    correct / len(test_data) > 0.82
), "Subject to random seed you should be able to get >82% accuracy"

AttributeError: 'MmBackward0' object has no attribute 'saved_tensors'